We based most of the calculations on the Python_Option_Pricing repository
https://github.com/dedwards25/Python_Option_Pricing/blob/master/GBS.ipynb

In [21]:
# import necessary libaries
import unittest
import math
import datetime
import numpy as np
from scipy.stats import norm
from scipy.stats import mvn

# Developer can toggle _DEBUG to True for more messages
# normally this is set to False
_DEBUG = False

In [85]:
_debug = print

In [4]:
# Black Scholes: stock Options (no dividend yield)
def black_scholes(option_type, fs, x, t, r, v):
    b = r
    return _gbs(option_type, fs, x, t, r, b, v)

In [56]:
# Inputs: option_type = "p" or "c", fs = price of underlying, x = strike, t = time to expiration, r = risk free rate
#         b = cost of carry, v = implied volatility
# Outputs: value, delta, gamma, theta, vega, rho
def _gbs(option_type, fs, x, t, r, b, v):
    print("Debugging Information: _gbs()")
    # -----------
    # Test Inputs (throwing an exception on failure)
    _gbs_test_inputs(option_type, fs, x, t, r, b, v)

    # -----------
    # Create preliminary calculations
    t__sqrt = math.sqrt(t)
    d1 = (math.log(fs / x) + (b + (v * v) / 2) * t) / (v * t__sqrt)
    d2 = d1 - v * t__sqrt

    if option_type == "c":
        # it's a call
        print("     Call Option")
        value = fs * math.exp((b - r) * t) * norm.cdf(d1) - x * math.exp(-r * t) * norm.cdf(d2)
        delta = math.exp((b - r) * t) * norm.cdf(d1)
        gamma = math.exp((b - r) * t) * norm.pdf(d1) / (fs * v * t__sqrt)
        theta = -(fs * v * math.exp((b - r) * t) * norm.pdf(d1)) / (2 * t__sqrt) - (b - r) * fs * math.exp(
            (b - r) * t) * norm.cdf(d1) - r * x * math.exp(-r * t) * norm.cdf(d2)
        vega = math.exp((b - r) * t) * fs * t__sqrt * norm.pdf(d1)
        rho = x * t * math.exp(-r * t) * norm.cdf(d2)
    else:
        # it's a put
        print("     Put Option")
        value = x * math.exp(-r * t) * norm.cdf(-d2) - (fs * math.exp((b - r) * t) * norm.cdf(-d1))
        delta = -math.exp((b - r) * t) * norm.cdf(-d1)
        gamma = math.exp((b - r) * t) * norm.pdf(d1) / (fs * v * t__sqrt)
        theta = -(fs * v * math.exp((b - r) * t) * norm.pdf(d1)) / (2 * t__sqrt) + (b - r) * fs * math.exp(
            (b - r) * t) * norm.cdf(-d1) + r * x * math.exp(-r * t) * norm.cdf(-d2)
        vega = math.exp((b - r) * t) * fs * t__sqrt * norm.pdf(d1)
        rho = -x * t * math.exp(-r * t) * norm.cdf(-d2)

    print("     d1= {0}\n     d2 = {1}".format(d1, d2))
    print("     delta = {0}\n     gamma = {1}\n     theta = {2}\n     vega = {3}\n     rho={4}".format(delta, gamma,
                                                                                                        theta, vega,
                                                                                                        rho))
    
    return value, delta, gamma, theta, vega, rho

In [57]:
class GBS_InputError(Exception):
    def __init__(self, mismatch):
        Exception.__init__(self, mismatch)

class _GBS_Limits:
    # An GBS model will return an error if an out-of-bound input is input
    MAX32 = 2147483248.0

    MIN_T = 1.0 / 1000.0  # requires some time left before expiration
    MIN_X = 0.01
    MIN_FS = 0.01

    # Volatility smaller than 0.5% causes American Options calculations
    # to fail (Number to large errors).
    # GBS() should be OK with any positive number. Since vols less
    # than 0.5% are expected to be extremely rare, and most likely bad inputs,
    # _gbs() is assigned this limit too
    MIN_V = 0.005

    MAX_T = 100
    MAX_X = MAX32
    MAX_FS = MAX32

    # Asian Option limits
    # maximum TA is time to expiration for the option
    MIN_TA = 0

    # This model will work with higher values for b, r, and V. However, such values are extremely uncommon. 
    # To catch some common errors, interest rates and volatility is capped to 100%
    # This reason for 1 (100%) is mostly to cause the library to throw an exceptions 
    # if a value like 15% is entered as 15 rather than 0.15)
    MIN_b = -1
    MIN_r = -1

    MAX_b = 1
    MAX_r = 1
    MAX_V = 5
      
def _test_option_type(option_type):
    if (option_type != "c") and (option_type != "p"):
        raise GBS_InputError("Invalid Input option_type ({0}). Acceptable value are: c, p".format(option_type))

def _gbs_test_inputs(option_type, fs, x, t, r, b, v):
    # -----------
    # Test inputs are reasonable
    _test_option_type(option_type)

    if (x < _GBS_Limits.MIN_X) or (x > _GBS_Limits.MAX_X):
        raise GBS_InputError(
            "Invalid Input Strike Price (X). Acceptable range for inputs is {1} to {2}".format(x, _GBS_Limits.MIN_X,
                                                                                               _GBS_Limits.MAX_X))

    if (fs < _GBS_Limits.MIN_FS) or (fs > _GBS_Limits.MAX_FS):
        raise GBS_InputError(
            "Invalid Input Forward/Spot Price (FS). Acceptable range for inputs is {1} to {2}".format(fs,
                                                                                                      _GBS_Limits.MIN_FS,
                                                                                                      _GBS_Limits.MAX_FS))

    if (t < _GBS_Limits.MIN_T) or (t > _GBS_Limits.MAX_T):
        raise GBS_InputError(
            "Invalid Input Time (T = {0}). Acceptable range for inputs is {1} to {2}".format(t, _GBS_Limits.MIN_T,
                                                                                             _GBS_Limits.MAX_T))

    if (b < _GBS_Limits.MIN_b) or (b > _GBS_Limits.MAX_b):
        raise GBS_InputError(
            "Invalid Input Cost of Carry (b = {0}). Acceptable range for inputs is {1} to {2}".format(b,
                                                                                                      _GBS_Limits.MIN_b,
                                                                                                      _GBS_Limits.MAX_b))

    if (r < _GBS_Limits.MIN_r) or (r > _GBS_Limits.MAX_r):
        raise GBS_InputError(
            "Invalid Input Risk Free Rate (r = {0}). Acceptable range for inputs is {1} to {2}".format(r,
                                                                                                       _GBS_Limits.MIN_r,
                                                                                                       _GBS_Limits.MAX_r))

    if (v < _GBS_Limits.MIN_V) or (v > _GBS_Limits.MAX_V):
        raise GBS_InputError(
            "Invalid Input Implied Volatility (V = {0}). \
            Acceptable range for inputs is {1} to {2}".format(v, _GBS_Limits.MIN_V, _GBS_Limits.MAX_V))

In [89]:
def convert_expiration_to_year(expiry_date):
    expiry_date = datetime.datetime.strptime(expiry_date,'%Y-%m-%dT%H:%M:%S.%fZ')
    time_diff_sec = (expiry_date - datetime.datetime.now()).total_seconds()
    return time_diff_sec/(3600*24*365)

In [67]:
black_scholes('c', 102, 100, 2, 0.05, 0.25)[0]

Debugging Information: _gbs()
     Call Option
     d1= 0.5156296959570099
     d2 = 0.16207630536373613
     delta = 0.6969434671865847
     gamma = 0.009685478594141343
     theta = -5.70233889678017
     vega = 50.383859646723266
     rho=102.13390675439865


20.02128027583231

In [68]:
'''
{"serumMarketAddress":"4raD3QLB9jKoUTzK48qfx2UdEaVHmc5xF8ZSaNK8vipk",
"exchange":"SOL","live":true,"strike":32,
"expiryDate":"2022-08-19T08:00:00.000Z","kind":"call",
"markPrice":"11.282046","delta":"0.01","impliedVolatility":"1.040269",
"vega":0.21037926345480487,"price":42.694415,
"confidence":0.010675,"priceStatus":1},
'''
underlying_price=42.69
strike=32
time_to_expiration=0.0182
risk_free_rate=0.
implied_volatility=1.040269
black_scholes('c', underlying_price, strike, time_to_expiration, 
              risk_free_rate,implied_volatility)

Debugging Information: _gbs()
     Call Option
     d1= 2.1239599091048387
     d2 = 1.9836199483631805
     delta = 0.9831632519529994
     gamma = 0.006979216813858741
     theta = -6.882089520892481
     vega = 0.2408108465795734
     rho=0.5686267572669556


(10.728010804612246,
 0.9831632519529994,
 0.006979216813858741,
 -6.882089520892481,
 0.2408108465795734,
 0.5686267572669556)

In [69]:
#{"serumMarketAddress":"6tFP43fEPZaW4Tf7QCpWDLzay6YgRQZ5Unt4bHqJpXeC","exchange":"BTC",
#"live":true,"strike":22000,"expiryDate":"2022-08-19T08:00:00.000Z","kind":"put","markPrice":"562.353902",
#"delta":"0.26","impliedVolatility":"1.039208","vega":1048.6306910377161,"price":23846.465,
#"confidence":13.84915651,"priceStatus":1},
underlying_price=23846.465
strike=22000
time_to_expiration=0.0182
risk_free_rate=0.
implied_volatility=1.039208
black_scholes('c', underlying_price, strike, time_to_expiration, 
              risk_free_rate,implied_volatility)

Debugging Information: _gbs()
     Call Option
     d1= 0.6449583314599002
     d2 = 0.5047615074437878
     delta = 0.7405229055368802
     gamma = 9.69216665271147e-05
     theta = -29760.74897797773
     vega = 1042.4200571958545
     rho=277.5319838112956


(2409.8434490617838,
 0.7405229055368802,
 9.69216665271147e-05,
 -29760.74897797773,
 1042.4200571958545,
 277.5319838112956)

In [55]:
# 7 day chart, 1-day
# delta vs underlying price
# plot greeks vs underlying_price

In [72]:
str1 = "2022-08-19T08:00:00.000Z"
convert_s_to_year(str1)

0.016043683853025112

In [77]:
def get_interest_free_rate(symbol):
    """ Values from solend """
    allowed_symbols =  ['ETH','BTC','SOL']
    if symbol not in allowed_symbols:
        raise Exception(f"Symbol {symbol} not in {allowed_symbols}")
    
    interest_free_rates = {
        'BTC':0.04/100,
        'ETH':0.85/100,
        'SOL':2.43/100 #2%
    }
    return interest_free_rates.get(symbol)

In [78]:
get_interest_free_rate('SOL')

0.024300000000000002

## Implied volatility

In [98]:
def euro_implied_vol(option_type, fs, x, t, r, q, cp):
    b = r - q
    return _gbs_implied_vol(option_type, fs, x, t, r, b, cp)

def euro_implied_vol_76(option_type, fs, x, t, r, cp):
    b = 0
    return _gbs_implied_vol(option_type, fs, x, t, r, b, cp)

def _gbs_implied_vol(option_type, fs, x, t, r, b, cp, precision=.00001, max_steps=100):
    return _newton_implied_vol(_gbs, option_type, x, fs, t, b, r, cp, precision, max_steps)

def _approx_implied_vol(option_type, fs, x, t, r, b, cp):
    _test_option_type(option_type)

    ebrt = math.exp((b - r) * t)
    ert = math.exp(-r * t)

    a = math.sqrt(2 * math.pi) / (fs * ebrt + x * ert)

    if option_type == "c":
        payoff = fs * ebrt - x * ert
    else:
        payoff = x * ert - fs * ebrt

    b = cp - payoff / 2
    c = (payoff ** 2) / math.pi

    v = (a * (b + math.sqrt(b ** 2 + c))) / math.sqrt(t)

    return v

def _newton_implied_vol(val_fn, option_type, x, fs, t, b, r, cp, precision=.00001, max_steps=100):
    # make sure a valid option type was entered
    _test_option_type(option_type)

    # Estimate starting Vol, making sure it is allowable range
    v = _approx_implied_vol(option_type, fs, x, t, r, b, cp)
    v = max(_GBS_Limits.MIN_V, v)
    v = min(_GBS_Limits.MAX_V, v)

    # Calculate the value at the estimated vol
    value, delta, gamma, theta, vega, rho = val_fn(option_type, fs, x, t, r, b, v)
    min_diff = abs(cp - value)

    _debug("-----")
    _debug("Debug info for: _Newton_ImpliedVol()")
    _debug("    Vinitial={0}".format(v))

    # Newton-Raphson Search
    countr = 0
    while precision <= abs(cp - value) <= min_diff and countr < max_steps:

        v = v - (value - cp) / vega
        if (v > _GBS_Limits.MAX_V) or (v < _GBS_Limits.MIN_V):
            _debug("    Volatility out of bounds")
            break

        value, delta, gamma, theta, vega, rho = val_fn(option_type, fs, x, t, r, b, v)
        min_diff = min(abs(cp - value), min_diff)

        # keep track of how many loops
        countr += 1
        _debug("     IVOL STEP {0}. v={1}".format(countr, v))

    
    # check if function converged and return a value
    if abs(cp - value) < precision:
        # the search function converged
        return v
    else:
        # if the search function didn't converge, try a bisection search
        return _bisection_implied_vol(val_fn, option_type, fs, x, t, r, b, cp, precision, max_steps)

In [87]:
_gbs_implied_vol('c', 92.45, 107.5, 0.0876712328767123, 0.00192960198828152, 0, 0.162619795863781)

Debugging Information: _gbs()
     Call Option
     d1= -0.5085315120480228
     d2 = -0.7484985492404215
     delta = 0.30548863872489856
     gamma = 0.01579875561031792
     theta = -44.33842839475904
     vega = 9.594378742128885
     rho=2.1397868562025115
-----
Debug info for: _Newton_ImpliedVol()
    Vinitial=0.8104440394140772
Debugging Information: _gbs()
     Call Option
     d1= -1.1278514382800449
     d2 = -1.2544698666367133
     delta = 0.12966939465866084
     gamma = 0.018038933606590198
     theta = -14.095763841775183
     vega = 5.7802971721916245
     rho=0.987872832356922
     IVOL STEP 1. v=0.4276301934726342
Debugging Information: _gbs()
     Call Option
     d1= -1.4889354959825585
     d2 = -1.5870017928994267
     delta = 0.0682406323735424
     gamma = 0.014521447458357229
     theta = -6.806814032955107
     vega = 3.6038945243829756
     rho=0.5301049726676658
     IVOL STEP 2. v=0.3312006796165272
Debugging Information: _gbs()
     Call Option
     d1= -1

0.30000001606213883

In [103]:
item = {"serumMarketAddress":"4raD3QLB9jKoUTzK48qfx2UdEaVHmc5xF8ZSaNK8vipk",
"exchange":"SOL","live":True,"strike":32,
"expiryDate":"2022-08-19T08:00:00.000Z","kind":"call",
"markPrice":"11.282046","delta":"0.01","impliedVolatility":"1.040269",
"vega":0.21037926345480487,"price":42.694415,
"confidence":0.010675,"priceStatus":1}

#    option_type = "p" or "c"
#    fs          = price of underlying
#    x           = strike
#    t           = time to expiration
#    v           = implied volatility
#    r           = risk free rate
#    q           = dividend payment
#    b           = cost of carry
# cp = Call or Put price
#def euro_implied_vol(option_type, fs, x, t, r, q, cp):
risk_free_rate = get_interest_free_rate(item['exchange'])
time_to_expiry = convert_expiration_to_year(item['expiryDate'] )
print('normal',euro_implied_vol('c',item['price'], item['strike'], time_to_expiry, risk_free_rate, 0., float(item['markPrice'])))
#print('76', euro_implied_vol_76('c',item['price'], item['strike'], time_to_expiry, risk_free_rate, float(item['markPrice'])))

Debugging Information: _gbs()
     Call Option
     d1= 0.8392115604541077
     d2 = 0.3561386578744979
     delta = 0.7993246994051226
     gamma = 0.013601747925972347
     theta = -181.47143611529282
     vega = 1.5142820682711198
     rho=0.3268030413615694
-----
Debug info for: _Newton_ImpliedVol()
    Vinitial=3.8208099028787905
Debugging Information: _gbs()
     Call Option
     d1= 1.162779647607471
     d2 = 0.8801222695301969
     delta = 0.8775405416521449
     gamma = 0.016814475778469003
     theta = -77.22529388196585
     vega = 1.0953250970296033
     rho=0.4144806198278293
     IVOL STEP 1. v=2.235646221330793
Debugging Information: _gbs()
     Call Option
     d1= 1.2667010006627222
     d2 = 1.0134545502945713
     delta = 0.8973688889584979
     gamma = 0.01654154458528399
     theta = -61.14326556415309
     vega = 0.9654255922533203
     rho=0.4318528318956922
     IVOL STEP 2. v=2.003023850579307
Debugging Information: _gbs()
     Call Option
     d1= 1.274407063

In [107]:
def get_implied_volatility_from_item(item):
    #    option_type = "p" or "c"
    #    fs          = price of underlying
    #    x           = strike
    #    t           = time to expiration
    #    v           = implied volatility
    #    r           = risk free rate
    #    q           = dividend payment
    #    b           = cost of carry
    # cp = Call or Put price
    #def euro_implied_vol(option_type, fs, x, t, r, q, cp):
    risk_free_rate = get_interest_free_rate(item['exchange'])
    time_to_expiry = convert_expiration_to_year(item['expiryDate'] )
    implied_vol = euro_implied_vol('c',item['price'], item['strike'], time_to_expiry, risk_free_rate, 0., float(item['markPrice']))
    #print('76', euro_implied_vol_76('c',item['price'], item['strike'], time_to_expiry, risk_free_rate, float(item['markPrice'])))
    return implied_vol

In [110]:
item.keys()

dict_keys(['serumMarketAddress', 'exchange', 'live', 'strike', 'expiryDate', 'kind', 'markPrice', 'delta', 'impliedVolatility', 'vega', 'price', 'confidence', 'priceStatus'])

In [129]:
{"confidence", "price2"} < item.keys()

False

In [113]:
item['kind']

'call'

In [115]:
'c' if item['kind'] == 'call1' else 'p'

'p'

In [116]:
item['kind'] not in ['call','put']

False

In [118]:
a = [{'a':1,'b':2},{'a':1,'b':3,'a':2,'b':4}]

In [120]:
import itertools

In [126]:
for key, group in itertools.groupby(a,lambda x: x['a']):
    print(str(key) + " :", list(group))

1 : [{'a': 1, 'b': 2}]
2 : [{'a': 2, 'b': 4}]
